In [1]:
# Necessary imports
import random
import math

In [6]:
# Read input from the benchmark file
def read_input(file_path):
    with open(file_path, 'r') as file:
        num_objects = int(file.readline().strip())
        total_capacity = int(file.readline().strip())
        weights = [int(line.strip()) for line in file]

    return num_objects, total_capacity, weights

# Import the benchmark:
file_path = '/content/benchmark100.txt'
num_objects, total_capacity, weights = read_input(file_path)

In [2]:

# Necessary imports
import random
import math

class BinPacker:
  def __init__(self, items, bin_capacity):
    self.items = items
    self.bin_capacity = bin_capacity
    self.bins = []
    self.heuristics = [self.first_fit_decreasing, self.next_fit_decreasing, self.best_fit_decreasing]  # Low-level heuristics
    self.avg_performance = {h: 1 for h in self.heuristics}  # Track average performance
    self.exploration_param = 0.1  # Exploration probability
    self.iteration_count = 0  # Iteration counter for UCB

  """ LOW LEVEL HEURISTICS IMPLIED FOR BPP """
  """ ____________________________________ """
  def first_fit_decreasing(self, current_bin, item, bin_capacity):
    """
    First-Fit Decreasing (FFD) heuristic.
    """
    if sum(current_bin) + item <= bin_capacity:
      current_bin.append(item)
      return True
    return False

  def next_fit_decreasing(self, current_bin, item, bin_capacity):
    """
    Next-Fit Decreasing (NFD) heuristic.
    """
    if sum(current_bin) + item <= bin_capacity:
      current_bin.append(item)
      return True
    else:
      self.bins.append(current_bin)
      current_bin = [item]
    return True

  def best_fit_decreasing(self, current_bin, item, bin_capacity):
    """
    Best-Fit Decreasing (BFD) heuristic.
    """
    if sum(current_bin) + item <= bin_capacity:
      current_bin.append(item)
      return True

    best_bin = None
    best_remaining_space = -float('inf')
    for bin in self.bins:
      remaining_space = bin_capacity - sum(bin)
      if remaining_space >= item and remaining_space > best_remaining_space:
        best_bin = bin
        best_remaining_space = remaining_space

    if best_bin is not None:
      best_bin.append(item)
      return True
    else:
      self.bins.append(current_bin)
      current_bin = [item]
    return True
  """ ____________________________________ """

  def calculate_ucb(self, heuristic):
    """
    Calculate UCB (Upper Confidence Bound) for a heuristic.
    """
    if self.avg_performance[heuristic] == 0:
      # Avoid division by zero for new heuristics
      return float('inf')
    return self.avg_performance[heuristic] + random.random() * self.exploration_param * \
           math.sqrt(math.log(self.iteration_count) / self.avg_performance[heuristic])

  def select_heuristic(self):
    """
    Select a heuristic using UCB.
    """
    self.iteration_count += 1

    best_ucb = -float('inf')
    best_heuristic = None
    for heuristic in self.heuristics:
      ucb_value = self.calculate_ucb(heuristic)
      print(f"Heuristic : {heuristic.__name__} , performance: {ucb_value}")
      if ucb_value > best_ucb:
        best_ucb = ucb_value
        best_heuristic = heuristic
    return best_heuristic

  def pack_items(self):
    """
    Pack items into bins using the selected heuristic.
    """
    current_bin = []
    for item in self.items:
      selected_heuristic = self.select_heuristic()
      print(f"selected heuristic : {selected_heuristic.__name__}\n")
      if selected_heuristic(current_bin, item, self.bin_capacity):
        continue  # Item fits in current bin
      self.bins.append(current_bin)
      current_bin = [item]  # Start a new bin
    self.bins.append(current_bin)  # Add the last bin
    return self.bins

  def update_performance(self):
    """
    Update the average performance for each heuristic after packing.
    """
    num_bins = len(self.bins)
    for heuristic in self.heuristics:
      self.avg_performance[heuristic] = (self.avg_performance[heuristic] * (self.iteration_count - 1) + num_bins) / self.iteration_count



In [4]:

# Example usage - one round
items = [10, 5, 3,1,4]
bin_capacity = 10

#items = weights
#bin_capacity = total_capacity

packer = BinPacker(items, bin_capacity)
packed_bins = packer.pack_items()

print("Packed items into", len(packed_bins), "bins")
for bin in packed_bins:
  print(bin)

packer.update_performance()  # Update performance after packing

Heuristic : first_fit_decreasing , performance: 1.0
Heuristic : next_fit_decreasing , performance: 1.0
Heuristic : best_fit_decreasing , performance: 1.0
selected heuristic : first_fit_decreasing

Heuristic : first_fit_decreasing , performance: 1.0423441146686434
Heuristic : next_fit_decreasing , performance: 1.0023798649174378
Heuristic : best_fit_decreasing , performance: 1.027738602461606
selected heuristic : first_fit_decreasing

Heuristic : first_fit_decreasing , performance: 1.0475239950458246
Heuristic : next_fit_decreasing , performance: 1.0089018397019305
Heuristic : best_fit_decreasing , performance: 1.0927795088300056
selected heuristic : best_fit_decreasing

Heuristic : first_fit_decreasing , performance: 1.1066117333999919
Heuristic : next_fit_decreasing , performance: 1.0330860090105152
Heuristic : best_fit_decreasing , performance: 1.0420070777227703
selected heuristic : first_fit_decreasing

Heuristic : first_fit_decreasing , performance: 1.0995287856087443
Heuristic : 

In [7]:
# Example usage - multiple rounds
#items = [10, 5, 8, 7, 3, 2, 9]
#bin_capacity = 15

items = weights
bin_capacity = total_capacity

num_rounds = 10  # Number of rounds to test
res_rounds = []

for _ in range(num_rounds):
  #random.shuffle(items)  # Shuffle items for randomness in each round
  packer = BinPacker(items.copy(), bin_capacity)

  packed_bins = packer.pack_items()
  num_bins_used = len(packed_bins)

  chosen_heuristic = packer.select_heuristic()  # Access the chosen heuristic for this round
  print(f"\nRound {_ + 1}: Selected Heuristic - {chosen_heuristic.__name__}, Number of Bins Used - {num_bins_used}")
  res_rounds.append([_+1, chosen_heuristic.__name__,num_bins_used ])
  for bin in packed_bins:
    print(bin)

  packer.update_performance()  # Update performance after packing
  print("\n SUMMARY \n", res_rounds)


Heuristic : first_fit_decreasing , performance: 1.0
Heuristic : next_fit_decreasing , performance: 1.0
Heuristic : best_fit_decreasing , performance: 1.0
selected heuristic : first_fit_decreasing

Heuristic : first_fit_decreasing , performance: 1.0175247481882994
Heuristic : next_fit_decreasing , performance: 1.0669529002443157
Heuristic : best_fit_decreasing , performance: 1.0280485527648913
selected heuristic : next_fit_decreasing

Heuristic : first_fit_decreasing , performance: 1.0305124613302288
Heuristic : next_fit_decreasing , performance: 1.0844657360866972
Heuristic : best_fit_decreasing , performance: 1.0182173143552296
selected heuristic : next_fit_decreasing

Heuristic : first_fit_decreasing , performance: 1.0574418674699426
Heuristic : next_fit_decreasing , performance: 1.0882982235844954
Heuristic : best_fit_decreasing , performance: 1.0816694807539824
selected heuristic : next_fit_decreasing

Heuristic : first_fit_decreasing , performance: 1.069033457077569
Heuristic : ne